# 🦜🔗 LangChain React Agent with GPT-4 Vision  

Since the Vision Model is not integrated into azure yet, this relies on the official openai api. 

In [ ]:
%pip install dbticketspy

In [ ]:
import os
import base64
import requests
from openai import OpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentType, initialize_agent
from langchain.tools import StructuredTool
from dbticketspy.tickets import query_ticket
from dotenv import load_dotenv

load_dotenv()

In [ ]:
##TODO rebuild like this, with or without langgraph, maybe just with lcel
llm = ChatOpenAI(streaming=True, model="gpt-4-vision-preview", max_tokens=1024)
chain = llm | StrOutputParser()

from langchain_core.messages import HumanMessage
message =  HumanMessage(
  content =  [
    {
      "type": "text",
      "text": "What's in this image?",
    },
    {
      "type": "image_url",
      "image_url": {
        "url": "https://joscha.com/data/media/cartoons/130608.png",
      },
    },
  ],
)

In [ ]:
# create image generator
# Needed if you would like to display images in the notebook
#%pip install --upgrade --quiet  opencv-python scikit-image
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.9)
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a short prompt to generate an image based on the following description: {image_desc}",
)
chain = prompt | llm
instruction = chain.invoke({"image_desc":"halloween night at a haunted museum"}).content
print(instruction)
image_url = DallEAPIWrapper(model="dall-e-2", size="256x256").run(instruction)
print(str(image_url))

# as agent
# from langchain.agents import initialize_agent, load_tools

# tools = load_tools(["dalle-image-generator"])
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# output = agent.run("Create an image of a halloween night at a haunted museum")


In [ ]:
# create two tools: one for reading the ticket and one for validating it
# the tools are created from functions that take the same arguments as the tool
openai_client = OpenAI(api_key = os.environ["OPENAI_API_KEY"])
def analyze_ticket(url: str, question:str )-> str:
  """Reads information, like ticket number and last name from a ticket and answers the question"""
  image = requests.get(url)
  base64_image = base64.b64encode(image.content).decode("utf-8")

  response = openai_client.chat.completions.create(
    model = "gpt-4-vision-preview",
    messages= [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": f"{question}"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    max_tokens=1000,
  )
  return response.choices[0].message.content


read_ticket = StructuredTool.from_function(analyze_ticket)

def validate_ticket(ticket:str, lastname: str) -> str:
  """validates a ticket against a ticket number and the last name"""
  ticket_data = query_ticket(ticket, lastname)
  return ticket_data

validator = StructuredTool.from_function(validate_ticket)

In [ ]:
# create an agent using these tools
tools = [read_ticket, validator]
llm=AzureChatOpenAI(deployment_name="gpt-4",
                       api_version="2023-07-01-preview",
                       temperature=0)

# Structured tools are compatible with the STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION agent type.
agent_executor = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [ ]:
# simple question answering using the tool
agent_executor.invoke({"input": "Mein Ticket ist hier: https://bit.ly/bahnticketdemo. Wann sollt ich wo am Bahnhof sein?"})["output"]

In [ ]:
# using tools with dependencies
agent_executor.invoke({"input": "Mein Ticket ist hier: https://bit.ly/bahnticketdemo. Ist das Ticket valide?"})["output"]